In the previous Notebooks i've established that using a Linear regression to predict Rank is my best bet.

This method is called  a pointwise ranking method. and the "learn to rank" Theory proposes more ranking methods that i'll try to explore in this norebook. 
    -Pairwise ranking
    -Listwise ranking

Let's start with Pairwise ranking there are various methods all from Microsoft researcher Burges
RankNet, LambdaRank, LambdaRankMart

In [1]:
import pandas as pd
from scipy.stats import spearmanr
import numpy as np
from utils.Dataset_modification import *
from utils.compute_model_performance import *

X_train = pd.read_csv(datasets+'X_train.csv') #datasets is defined in utils
Y_train=pd.read_csv(datasets+'Y_train.csv')
Y_train['Rank']= Y_train['TARGET'].rank()


In [3]:
df=create_pairwise_dataset(Y_train)
print(df)
remade=reconstruct_original_dataset(df)
remade

{1054.0: 807, 2049.0: 532, 1924.0: 427, 297.0: 343, 1101.0: 593, 1520.0: 1351, 1546.0: 1057, 1069.0: 968, 1323.0: 906, 1618.0: 747, 1507.0: 260, 819.0: 495, 918.0: 1062, 283.0: 684, 158.0: 1250, 1821.0: 1247, 1263.0: 617, 240.0: 242, 1897.0: 987, 964.0: 47, 519.0: 782, 1199.0: 165, 1482.0: 249, 1104.0: 993, 971.0: 787, 583.0: 883, 1715.0: 1133, 2004.0: 917, 2129.0: 1388, 837.0: 308, 1866.0: 1206, 902.0: 401, 1059.0: 451, 1659.0: 1161, 1409.0: 1045, 304.0: 1067, 1834.0: 1093, 269.0: 231, 970.0: 704, 935.0: 759, 606.0: 216, 732.0: 96, 518.0: 1397, 1499.0: 30, 1802.0: 555, 1675.0: 911, 2094.0: 588, 1813.0: 485, 315.0: 1268, 541.0: 1196, 1106.0: 508, 1119.0: 250, 2013.0: 1081, 464.0: 1303, 2076.0: 693, 760.0: 207, 415.0: 1225, 8.0: 42, 1113.0: 233, 1709.0: 1183, 99.0: 407, 936.0: 480, 1560.0: 197, 263.0: 170, 955.0: 830, 912.0: 958, 114.0: 594, 1432.0: 734, 254.0: 256, 1493.0: 1203, 549.0: 1139, 1052.0: 293, 1742.0: 1091, 2107.0: 20, 1811.0: 1483, 21.0: 991, 526.0: 1076, 128.0: 265, 612.0:

,ID,TARGET
0,0.0,455
1,2.0,778
2,3.0,364
3,4.0,810
4,5.0,589
...,...,...
1489,2142.0,1357
1490,2143.0,1314
1491,2144.0,571
1492,2145.0,434


In [13]:
print(len(df))
len(Y_train)*(len(Y_train)-1)/2 # We obtain a dataset of length n(n-1)/2

1115271


1115271.0

In [8]:
sorted=remade.sort_values(by=['ID'], key=lambda x: x.map({k: i for i, k in enumerate(Y_train['ID'].tolist())}))
sorted['ID'].tolist()==Y_train['ID'].tolist()

True

In [9]:
from scipy.stats import spearmanr
print("TARGET AND reconstructed Rank :" ,spearmanr(sorted["TARGET"], Y_train["TARGET"]).correlation)


TARGET AND reconstructed Rank : 1.0


Predicting Pairwise ranking accurately -> predicting Ranking accurately -> high spearman score

meaning our method of creating pairwise ranking and prediction a pairwise ranking ID by ID might help determine a more accurate ranking. Now let's Get a  X_train_pairwise that matches our Y_train_Pairwise

In [3]:
X_train_pairwise=get_training_pairwise(X_train)

In [4]:
X_train_pairwise

,ID1,ID2,DAY_ID1,DAY_ID2,COUNTRY1,COUNTRY2,DE_CONSUMPTION1,DE_CONSUMPTION2,FR_CONSUMPTION1,FR_CONSUMPTION2,...,DE_TEMP1,DE_TEMP2,FR_TEMP1,FR_TEMP2,GAS_RET1,GAS_RET2,COAL_RET1,COAL_RET2,CARBON_RET1,CARBON_RET2
0,1054,2049,206,501,FR,FR,0.210099,-0.022399,-0.427458,-1.003452,...,-1.069070,0.437419,-0.063404,1.831241,0.339041,-0.659091,0.124552,0.047114,-0.002445,-0.490365
1,1054,1924,206,687,FR,FR,0.210099,1.395035,-0.427458,1.978665,...,-1.069070,0.684884,-0.063404,0.114836,0.339041,0.535974,0.124552,0.743338,-0.002445,0.204952
2,1054,297,206,720,FR,DE,0.210099,-0.983324,-0.427458,-0.849198,...,-1.069070,0.350938,-0.063404,-0.417514,0.339041,0.911652,0.124552,-0.296168,-0.002445,1.073948
3,1054,1101,206,818,FR,FR,0.210099,0.143807,-0.427458,-0.617038,...,-1.069070,0.614338,-0.063404,0.729495,0.339041,0.245109,0.124552,1.526606,-0.002445,2.614378
4,1054,1520,206,467,FR,FR,0.210099,-0.295296,-0.427458,-0.765120,...,-1.069070,0.102046,-0.063404,0.472708,0.339041,0.891049,0.124552,0.861408,-0.002445,1.124457
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1115266,1674,1454,887,1133,FR,FR,1.618582,0.560689,1.752840,-0.343777,...,NaN,1.215528,NaN,1.338708,0.932633,0.962812,-0.085690,-5.392852,0.356356,-0.843812
1115267,1674,359,887,1118,FR,DE,1.618582,0.226730,1.752840,-0.711005,...,NaN,-0.120703,NaN,0.391491,0.932633,1.059829,-0.085690,-0.326640,0.356356,-0.566724
1115268,748,1454,1083,1133,DE,FR,0.856399,0.560689,0.489199,-0.343777,...,0.256338,1.215528,0.402316,1.338708,-1.112899,0.962812,-0.237835,-5.392852,0.067152,-0.843812
1115269,748,359,1083,1118,DE,DE,0.856399,0.226730,0.489199,-0.711005,...,0.256338,-0.120703,0.402316,0.391491,-1.112899,1.059829,-0.237835,-0.326640,0.067152,-0.566724
